In [1]:
!pip install transformers datasets accelerate


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch


In [3]:
from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories", split="train[:3000]")
print(dataset[0]["text"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.


In [4]:
model_name = "gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue

model = GPT2LMHeadModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # For causal language modeling, labels are typically the input_ids themselves
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [6]:
training_args = TrainingArguments(
    output_dir="./gpt2-story-model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_strategy="no",
    logging_steps=50,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none" # Disable reporting to Weights & Biases
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,2.399200
100,2.222500
150,2.128500
200,2.095700
250,2.113500
300,2.055100
350,2.053900
400,2.023000
450,1.993700
500,2.003300


TrainOutput(global_step=2250, training_loss=1.9195194769965278, metrics={'train_runtime': 377.649, 'train_samples_per_second': 23.832, 'train_steps_per_second': 5.958, 'total_flos': 587907072000000.0, 'train_loss': 1.9195194769965278, 'epoch': 3.0})

In [10]:
def generate_story(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")

    # Move inputs to the same device as the model
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=180,
        min_length=60,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_story("In a dark forest"))

In a dark forest, there was an old man with long hair. He had never seen something so funny in the wild before!
 He smiled and bowed his head high as he watched everyone walking by him like ghosts or having fun under it all day."Blessing," said Gary Dyer when I asked who to speak of today!" The bald guy stopped for just short seconds because both sides were waiting on top of their faces together until they'd reached its destination at last - which meant "the new world we imagined".â€It seemed that one would soon come full circle after being given this special task over years?" added Jacky while nodding along happily behind her back"I wish luck will always be mine again!", whispered John Masella upon crossing through into my garden too early...
